In [54]:
import pandas as pd
import numpy as np
import os
import PIL
from keras.preprocessing.image import load_img,img_to_array
from keras import Model,Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
#conda install -c anaconda keras


In [134]:
import os

hot_dog_image_dir = 'train/hot_dog/'
hot_dog_paths = [''.join(hot_dog_image_dir+filename) for filename in 
                            os.listdir(hot_dog_image_dir)]

not_hot_dog_image_dir = 'train/not_hot_dog/'
not_hot_dog_paths = [''.join(not_hot_dog_image_dir+filename) for filename in
                            os.listdir(not_hot_dog_image_dir)]

image_paths_train = hot_dog_paths + not_hot_dog_paths
y_dog=np.ones((len(hot_dog_paths),1),dtype=int)
y_not=np.zeros((len(not_hot_dog_paths),1),dtype=int)
y_train=np.concatenate((y_dog, y_not), axis=0)
# print(image_paths_train)
# print(y_train)

###############################################################################################################
hot_dog_image_dir_test = 'test/hot_dog/'
hot_dog_paths_test = [''.join(hot_dog_image_dir_test+filename) for filename in 
                            os.listdir(hot_dog_image_dir_test)]

not_hot_dog_image_dir_test = 'test/not_hot_dog/'
not_hot_dog_paths_test = [''.join(not_hot_dog_image_dir_test+filename) for filename in
                            os.listdir(not_hot_dog_image_dir_test)]

image_paths_test = hot_dog_paths_test + not_hot_dog_paths_test
y_dog_test=np.ones((len(hot_dog_paths_test),1),dtype=int)
y_dog_not=np.zeros((len(not_hot_dog_paths_test),1),dtype=int)
y_test=np.concatenate((y_dog_test, y_dog_not), axis=0)
# print(image_paths_test)
# print(y_test)
# print(y_test.shape)

In [135]:
def loadpath(image_paths,xsize=16,ysize=16):
    X=[]
    for location in image_paths:
        img=load_img(location,target_size=(xsize,ysize))
        sx=img_to_array(img) #(512, 382, 3)
#         print(x.shape)
#         sx=sx.reshape((1,)+sx.shape) #(1, 512, 382, 3)
#         print(sx.shape)
#         print(sx)
        X.append(sx)
        input_shape=sx.shape   
    return input_shape,np.array(X)

In [136]:
input_shape,x_train=loadpath(image_paths_train)
input_shape,x_test=loadpath(image_paths_test)

# from keras.utils import to_categorical
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# y_train=y_train.transpose()
# y_test=y_test.transpose()

# y_train=np.matrix(y_train)
# y_test=np.matrix(y_test)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(498, 16, 16, 3)
(500, 16, 16, 3)
(498, 1)
(500, 1)


In [58]:
# from keras.preprocessing.image import ImageDataGenerator

# def weirdfilters(x,num_gen,ImageDataGenerator(
#             rotation_range=40,
#             width_shift_range=0.2,
#             height_shift_range=0.2,
#             rescale=1./255,
#             shear_range=0.2,
#             zoom_range=0.2,
#             horizontal_flip=True,
#             fill_mode='nearest')
                 
                 
#     datagen = ImageDataGenerator(
#             rotation_range=40,
#             width_shift_range=0.2,
#             height_shift_range=0.2,
#             rescale=1./255,
#             shear_range=0.2,
#             zoom_range=0.2,
#             horizontal_flip=True,
#             fill_mode='nearest')
#     count=0
#     while(count!=x):
#         count+=1
#         datagen.flow(x,batch_size=1,save_to_dir'preview',save_prefix='cat',save_format='jpeg')
        
    
        
#     return datagen

In [139]:
from keras import Model,Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
# Basic model1

M1 = Sequential()

M1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))

M1.add(Conv2D(64, (3, 3), activation='relu'))

M1.add(MaxPooling2D(pool_size=(3, 3)))

M1.add(Flatten())

M1.add(Dense(128, activation='relu'))

M1.add(Dense(1, activation='sigmoid'))

M1.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        896       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               131200    
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 150,721
Trainable params: 150,721
Non-trainable params: 0
_________________________________________________________________


In [140]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.val_acc = []
        self.acc = []
        
    def on_batch_end(self, batch, logs={}):
        self.loss.append(logs.get('loss'))
        
    def on_epoch_end(self, epoch, logs):
        self.val_acc.append(logs.get('val_acc'))
        self.acc.append(logs.get('acc'))

history_cb = LossHistory()

In [142]:
import keras
batch_size=12
epochs=1
M1.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adadelta(lr=0.01),metrics=['accuracy'])

M1.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test),callbacks=[history_cb])

score = M1.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 498 samples, validate on 500 samples
Epoch 1/1
498/498 [==============================] - 3s 6ms/step - loss: 0.5092 - acc: 0.4880 - val_loss: 0.4963 - val_acc: 0.4980
('Test loss:', 0.4963350520502165)
('Test accuracy:', 0.4979999995231628)


In [1]:
M1.save('Skynet_M1')
model = keras.models.load_model('')

NameError: name 'M1' is not defined

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

model1Hist = history_cb
xVal = np.arange(1, 21, 1)
xValLoss = np.arange(1, 187, 1)
plt.plot(xVal, model1Hist.acc)
plt.plot(xVal, model1Hist.val_acc)
plt.title('Model 1 (without Batch Normalization)')
plt.legend(("Training", "Testing"), loc="best")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')